In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import cv2
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
path = '/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/'
os.listdir(path)

In [ ]:
train_data = pd.read_csv(path+'train.csv')
sample_data= pd.read_csv(path+'sample_submission.csv')

In [ ]:
print('Number train images:', len(train_data.index))
print('Number test images:', len(sample_data.index))

In [ ]:
plt.figure(figsize=(26, 8))
sns.countplot(x="class_name", data=train_data)
plt.title("Class Name Distribution")
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))
sns.countplot(x="class_id", data=train_data)
plt.title("Class ID Distribution")
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))
sns.countplot(x="rad_id", data=train_data)
plt.title("RAD ID Distribution")
plt.show()



In [ ]:
# ref kernel: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
import pydicom 
import numpy as np
from pydicom.pixel_data_handlers.util import apply_voi_lut

def read_xray(path, voi_lut = True, fix_monochrome = True):
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

In [ ]:
def plot_pixel_array(data, figsize=(8,8)):
    plt.figure(figsize=figsize)
    plt.imshow(data, cmap=plt.cm.bone)
    plt.show()

In [ ]:
train_dir = "../input/vinbigdata-chest-xray-abnormalities-detection/train"
test_dir = "../input/vinbigdata-chest-xray-abnormalities-detection/test"

train_files = os.listdir(train_dir)
test_files = os.listdir(test_dir)

In [ ]:
print("Examining train images...")
for _ in range(5):
    fn = train_files[np.random.randint(0, len(train_files))]
    file_path = os.path.join(train_dir, fn)
    data = read_xray(file_path)
    plot_pixel_array(data)

Checking the Localizations

In [ ]:

for _ in range(10):
    idx = np.random.randint(0, len(train_files))
    img_id = train_data.loc[idx, 'image_id']
    img = read_xray(os.path.join(train_dir, img_id+".dicom"))
    figg= plt.figure(figsize=(6,6))
    figg.add_subplot(1,1,1)
    plt.imshow(img, cmap='gray')
    plt.title(train_data.loc[idx, 'class_name'])
    
    if train_data.loc[idx, 'class_name'] != 'No finding':
        bbox = [train_data.loc[idx, 'x_min'],
                train_data.loc[idx, 'y_min'],
                train_data.loc[idx, 'x_max'],
                train_data.loc[idx, 'y_max']]
        
        patch = matplotlib.patches.Rectangle((bbox[0], bbox[1]),
                              bbox[2]-bbox[0],
                              bbox[3]-bbox[1],
                              ec='r', fc='none', lw=2.)
        ax = plt.gca()
        ax.add_patch(patch)

Output

In [ ]:
sample_data.to_csv('submission1.csv',index= False)

In [ ]:
output = pd.read_csv('submission1.csv',)
output